In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import matplotlib.pyplot as plt

In [2]:
grid = gpd.read_file('grid_1mile.json').assign(
    idx = lambda x: 'hex-' + x.index.astype(str)
)

In [3]:
dat_points = pd.read_csv('priority1calls.csv').assign(
    geometry = lambda x: x.apply(
        lambda row: Point(
            row['Incident_Longitude'],
            row['Incident_Latitude']
        ), axis=1
    )
)

In [12]:
# Added by Steve
dat_points.head()

,Incident_Code,Key_Month,Incident_Date,Incident_Priority,Incident_Problem,Incident_Zip_Code,Incident_ESD,Incident_Zone,Incident_Longitude,Incident_Latitude,...,First_Unit_Arrived_Ambulance,Time_First_Departed_Scene,Time_First_At_Destination,Incident_Dispositions,Count_Responses,Flag_Other_Agency_Call_Taker (1=Yes),TimeInterval_CallReceived_to_FirstArrived (Sec),"Response_Interval_Goal_Met (1=Y; "" ""=N)",Incident_Response_Interval_Goal,geometry
0,11005617,201301,2013-01-01 00:00:00,1,Unconscious Pri 1,78748.0,City of Austin,City of Austin,-97.790374,30.170623,...,M28,NaN,NaN,Refusal,1,0,606,0,City of Austin - Priority 1 (09:59 / 90%),POINT (-97.790374 30.170623)
1,11005641,201301,2013-01-01 00:00:00,1,Diabetic Pri 1,78704.0,City of Austin,City of Austin,-97.767637,30.229450,...,M01,2013-01-01 01:38:17,2013-01-01 01:42:58,South Austin,1,0,177,1,City of Austin - Priority 1 (09:59 / 90%),POINT (-97.76763699999999 30.22945)
2,11005807,201301,2013-01-01 00:00:00,1,Unconscious Pri 1,78753.0,City of Austin,City of Austin,-97.664745,30.398738,...,M29,2013-01-01 02:10:59,2013-01-01 02:25:29,North Austin,1,0,512,1,City of Austin - Priority 1 (09:59 / 90%),POINT (-97.664745 30.398738)
3,11005817,201301,2013-01-01 00:00:00,1,Unconscious Pri 1,78705.0,City of Austin,City of Austin,-97.737234,30.300343,...,M13,2013-01-01 02:23:52,2013-01-01 02:30:17,Seton,1,0,622,0,City of Austin - Priority 1 (09:59 / 90%),POINT (-97.737234 30.300343)
4,11006522,201301,2013-01-01 00:00:00,1,Respiratory Pri 1,78660.0,ESD02,Travis County,-97.634920,30.469386,...,M20,2013-01-01 04:06:18,2013-01-01 04:12:58,Saint Davids Pflugerville FSED,1,0,418,1,Travis County - Priority 1 (11:59 / 90%),POINT (-97.63491999999999 30.469386)


In [15]:
# Added by Steve
grid.head()

,geometry,idx
0,"POLYGON ((-98.36871000438374 29.9004898966998,...",hex-0
1,POLYGON ((-98.36871000438374 29.92555814224629...,hex-1
2,POLYGON ((-98.36871000438374 29.95062638779278...,hex-2
3,POLYGON ((-98.36871000438374 29.97569463333928...,hex-3
4,POLYGON ((-98.36871000438374 30.00076287888577...,hex-4


In [5]:
dat_points = gpd.GeoDataFrame(
    dat_points
)

dat_points.crs = grid.crs

In [6]:
dat_points_grid = gpd.sjoin(
    dat_points,
    grid,
    how='left',
    op='within'
)

In [7]:
# Added by Steve
dat_points_grid.head()

,Incident_Code,Key_Month,Incident_Date,Incident_Priority,Incident_Problem,Incident_Zip_Code,Incident_ESD,Incident_Zone,Incident_Longitude,Incident_Latitude,...,Time_First_At_Destination,Incident_Dispositions,Count_Responses,Flag_Other_Agency_Call_Taker (1=Yes),TimeInterval_CallReceived_to_FirstArrived (Sec),"Response_Interval_Goal_Met (1=Y; "" ""=N)",Incident_Response_Interval_Goal,geometry,index_right,idx
0,11005617,201301,2013-01-01 00:00:00,1,Unconscious Pri 1,78748.0,City of Austin,City of Austin,-97.790374,30.170623,...,NaN,Refusal,1,0,606,0,City of Austin - Priority 1 (09:59 / 90%),POINT (-97.790374 30.170623),863.0,hex-863
1,11005641,201301,2013-01-01 00:00:00,1,Diabetic Pri 1,78704.0,City of Austin,City of Austin,-97.767637,30.229450,...,2013-01-01 01:42:58,South Austin,1,0,177,1,City of Austin - Priority 1 (09:59 / 90%),POINT (-97.76763699999999 30.22945),865.0,hex-865
2,11005807,201301,2013-01-01 00:00:00,1,Unconscious Pri 1,78753.0,City of Austin,City of Austin,-97.664745,30.398738,...,2013-01-01 02:25:29,North Austin,1,0,512,1,City of Austin - Priority 1 (09:59 / 90%),POINT (-97.664745 30.398738),1049.0,hex-1049
3,11005817,201301,2013-01-01 00:00:00,1,Unconscious Pri 1,78705.0,City of Austin,City of Austin,-97.737234,30.300343,...,2013-01-01 02:30:17,Seton,1,0,622,0,City of Austin - Priority 1 (09:59 / 90%),POINT (-97.737234 30.300343),939.0,hex-939
4,11006522,201301,2013-01-01 00:00:00,1,Respiratory Pri 1,78660.0,ESD02,Travis County,-97.634920,30.469386,...,2013-01-01 04:12:58,Saint Davids Pflugerville FSED,1,0,418,1,Travis County - Priority 1 (11:59 / 90%),POINT (-97.63491999999999 30.469386),1088.0,hex-1088


In [8]:
breakdown_priority = dat_points_grid[[
    'idx',
    'Incident_Priority'
]].pivot_table(
    index='idx',
    columns='Incident_Priority',
    aggfunc=pd.Series.count
).apply(
    lambda x: round(
        (x / sum(x)) * 100,
        2
    ), axis=1
).fillna(0).rename(
    columns = lambda x: 'priority-' + str(x)
)

In [11]:
# Added by Steve
breakdown_priority.head()

Incident_Priority,priority-1
idx,
hex-1000,100.0
hex-1001,100.0
hex-1002,100.0
hex-1003,100.0
hex-1004,100.0


In [12]:
breakdown_seconds = dat_points_grid.groupby('idx')['Incident_Priority'].agg([
        pd.np.size
    ]
)

In [13]:
# Added by steve
breakdown_seconds.head()

,size
idx,
hex-1000,11
hex-1001,10
hex-1002,17
hex-1003,13
hex-1004,42


In [8]:
breakdown_goal = dat_points_grid.groupby('idx')[
    'Response_Interval_Goal_Met (1=Y; " "=N)'
].agg([
    sum,
    pd.Series.count
]).rename(
    columns={
        'sum': 'goal_met',
        'count': 'total_count'
    }
).assign(
    pct_goal_met = lambda x: round(
        (x['goal_met'] / x['total_count']) * 100,
        2
    )
)

In [9]:
grid_merge = grid.set_index('idx').join(
    breakdown_seconds.join(
        breakdown_priority
    ).join(
        breakdown_goal
    )
).fillna(0).query('size > 0').rename(
    columns = lambda x: str(x)
)

In [10]:
grid_merge.to_file(
    'points-grid_1mile.geojson',
    driver='GeoJSON'
)